In [1]:
from __future__ import print_function
import os
import imp
import logging
imp.reload(logging)
logging.basicConfig(filename=r'C:\Users\luiz\Projects\vradam-mxnet\example.log',filemode='w',level=logging.DEBUG)
logging.debug('This message should go to the log file')
import mxnet as mx
import requests as rq
import pandas as pd
import time, datetime
import numpy as np
import re

In [2]:
def download_svrg_module():

    try:
        os.makedirs('./svrg_optimization')
        init_svrg = rq.get(
            'https://raw.githubusercontent.com/apache/incubator-mxnet/master/python/mxnet/contrib/svrg_optimization/__init__.py')
        with open('./svrg_optimization/__init__.py', 'w') as file_init:
            file_init.write(init_svrg.text)

        modu_svrg = rq.get(
            'https://raw.githubusercontent.com/apache/incubator-mxnet/master/python/mxnet/contrib/svrg_optimization/svrg_module.py')
        with open('./svrg_optimization/svrg_module.py', 'w') as file_modu:
            file_modu.write(modu_svrg.text)

        opti_svrg = rq.get(
            'https://raw.githubusercontent.com/apache/incubator-mxnet/master/python/mxnet/contrib/svrg_optimization/svrg_optimizer.py')
        with open('./svrg_optimization/svrg_optimizer.py', 'w') as file_opti:
            file_opti.write(opti_svrg.text)

    except FileExistsError:
        pass

In [7]:
try:
    from svrg_optimization.svrg_module import SVRGModule
except ModuleNotFoundError:
    download_svrg_module()
    from svrg_optimization.svrg_module import SVRGModule

In [8]:
class LogisticRegression(object):

    def __init__(self, dataset, epochs, context, batch_size, is_svrg=True, optimizer='sgd', hyperparams=None, **kwargs):
        self.dataset = dataset.lower()
        self.epochs = epochs
        self.context = context
        self.hyperparams = hyperparams
        self.batch_size = batch_size
        self.is_svrg = is_svrg
        self.optimizer = optimizer
        self.train_iter = None
        self.test_iter = None

        self.define_dataset()
        self.log_path = r'evals.log'
        import logging
        imp.reload(logging)
        logging.basicConfig(filename=self.log_path,filemode='w',level=logging.DEBUG)
        #logging.debug('This message should go to the log file')
        #self.logger = logging.getLogger()
        #self.logger.setLevel(logging.DEBUG)  # logging to stdout

        # return super().__init__(dataset, epochs, context, hyperparams, **kwargs)

    def define_dataset(self):
        if self.dataset == 'mnist' or self.dataset is None:
            # define dataset and dataloader
            mnist = mx.test_utils.get_mnist()

            self.train_iter = mx.io.NDArrayIter(mnist['train_data'],
                                                mnist['train_label'],
                                                self.batch_size, shuffle=True, data_name='data',
                                                label_name='softmax_label')
            self.test_iter = mx.io.NDArrayIter(mnist['test_data'],
                                               mnist['test_label'],
                                               self.batch_size * 2, shuffle=False, data_name='data',
                                               label_name='softmax_label')

        elif self.dataset == 'cifar-10':
            mx.test_utils.get_cifar10()

    def process_log(self):
        # define the paths to the training logs
        logs = [
            (0, self.log_path)
        ]

        # initialize the list of train rank-1 and rank-5 accuracies, along
        # with the training loss
        (trainRank1, trainRank5, trainLoss) = ([], [], [])

        # initialize the list of validation rank-1 and rank-5 accuracies,
        # along with the validation loss
        (valRank1, valRank5, valLoss) = ([], [], [])

        # loop over the training logs
        for (i, (endEpoch, p)) in enumerate(logs):
            # load the contents of the log file, then initialize the batch
            # lists for the training and validation data
            rows = open(p).read().strip()
            (bTrainRank1, bTrainRank5, bTrainLoss) = ([], [], [])
            (bValRank1, bValRank5, bValLoss) = ([], [], [])

            # grab the set of training epochs
            epochs = set(re.findall(r'Epoch\[(\d+)\]', rows))
            epochs = sorted([int(e) for e in epochs])

            # loop over the epochs
            for e in epochs:
                # find all rank-1 accuracies, rank-5 accuracies, and loss
                # values, then take the final entry in the list for each
                s = r'Epoch\[' + str(e) + '\].*accuracy=([0]*\.?[0-9]+)'

                rank1 = re.findall(s, rows)[-2]
                #print(rank1)
                #s = r'Epoch\[' + str(e) + '\].*top_k_accuracy_5=([0]*\.?[0-9]+)'
                #rank5 = re.findall(s, rows)[-2]
                s = r'Epoch\[' + str(e) + '\].*cross-entropy=([0-9]*\.?[0-9]+)'
                loss = re.findall(s, rows)[-2]

                # update the batch training lists
                bTrainRank1.append(float(rank1))
                #bTrainRank5.append(float(rank5))
                bTrainLoss.append(float(loss))

            # extract the validation rank-1 and rank-5 accuracies for each
            # epoch, followed by the loss
            bValRank1 = re.findall(r'Validation-accuracy=(.*)', rows)
            #bValRank5 = re.findall(r'Validation-top_k_accuracy_5=(.*)', rows)
            bValLoss = re.findall(r'Validation-cross-entropy=(.*)', rows)

            # convert the validation rank-1, rank-5, and loss lists to floats
            bValRank1 = [float(x) for x in bValRank1]
            #bValRank5 = [float(x) for x in bValRank5]
            bValLoss = [float(x) for x in bValLoss]
            
            df = pd.DataFrame({'val_acc': bValRank1,
                               'val_loss': bValLoss,
                               'train_acc': bTrainRank1,
                               'train_loss': bTrainLoss})
            
            return df
            
    
    def run_logistic(self, update_freq=2):

        # data input and formatting
        data = mx.sym.var('data')  # (bs, 1, 28, 28) - MNIST
        label = mx.sym.var('softmax_label')
        data = mx.sym.Flatten(data)  # (bs, 28*28) - MNIST

        # logistic regression network
        fc = mx.sym.FullyConnected(data, num_hidden=10, name='fc')
        logist = mx.sym.SoftmaxOutput(fc, label=label, name='softmax')
    
        # metrics and eval
        metric_list = [mx.metric.Accuracy(output_names=['softmax_output'], label_names=['softmax_label']),
                       mx.metric.CrossEntropy(output_names=['softmax_output'], label_names=['softmax_label'])]
        eval_metrics = mx.metric.CompositeEvalMetric(metric_list)

        # create and 'compile' network
        if self.is_svrg:
            model = SVRGModule(symbol=logist,
                               data_names=['data'],
                               label_names=['softmax_label'],
                               context=self.context,
                               update_freq=update_freq)
        else:
            model = mx.mod.Module(logist,
                                  data_names=['data'],
                                  label_names=['softmax_label'],
                                  context=self.context)

        model.bind(data_shapes=self.train_iter.provide_data,
                   label_shapes=self.train_iter.provide_label)
        model.init_params()
        model.init_optimizer(kvstore='local',
                             optimizer=self.optimizer,
                             optimizer_params=self.hyperparams)

        timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
        model.fit(self.train_iter,  # train data
                  eval_data=self.test_iter,  # validation data
                  # optimizer='sgd',  # use SGD to train
                  # optimizer_params = (('learning_rate', lr),),  # use fixed learning rate
                  eval_metric=eval_metrics,  # report accuracy during training
                  # output progress for each 500 data batches
                  batch_end_callback=mx.callback.Speedometer(self.batch_size, 1000),
                  #epoch_end_callback=mx.callback.log_train_metric, #self.log_call,#mx.callback.do_checkpoint('logistic', 10),
                  num_epoch=self.epochs)  # train for at most <epochs> dataset passes
        
        self.df_out = self.process_log()
        print(timestamp)
        self.df_out['timestamp'] = pd.Series(timestamp, index=self.df_out.index)
        self.df_out['optimizer'] = pd.Series(self.optimizer, index=self.df_out.index)
        self.df_out['is_svrg'] = pd.Series(self.is_svrg, index=self.df_out.index)
        self.df_out['batch_size'] = pd.Series(self.batch_size, index=self.df_out.index)
        self.df_out['hyperparams'] = pd.Series(str(self.hyperparams), index=self.df_out.index)
        self.df_out['epoch'] = pd.Series(self.df_out.index + 1, index=self.df_out.index)
        self.df_out['update_freq'] = pd.Series(update_freq, index=self.df_out.index)
        
        #print(self.df_out)
        #val_score = model.score(self.train_iter, 'acc')[0][1]
    
        #train_score = model.score(self.train_iter, 'acc')[0][1]
        #print(val_score, train_score)

        # TODO: Define function returns
        # IDEA: pandas df with all class params, timestamp, and scores. Save this to disk
        # lr_params_svrg.update({lr: [val_score, train_score]})

        #return train_score

In [9]:
class NeuralNetwork(object):

    def __init__(self, dataset, epochs, context, batch_size, is_svrg=True, optimizer='sgd', hyperparams=None, **kwargs):
        self.dataset = dataset.lower()
        self.epochs = epochs
        self.context = context
        self.hyperparams = hyperparams
        self.batch_size = batch_size
        self.is_svrg = is_svrg
        self.optimizer = optimizer
        self.train_iter = None
        self.test_iter = None

        self.define_dataset()
        self.log_path = r'evals.log'
        import logging
        imp.reload(logging)
        logging.basicConfig(filename=self.log_path,filemode='w',level=logging.DEBUG)
        #logging.debug('This message should go to the log file')
        #self.logger = logging.getLogger()
        #self.logger.setLevel(logging.DEBUG)  # logging to stdout

        # return super().__init__(dataset, epochs, context, hyperparams, **kwargs)

    def define_dataset(self):
        if self.dataset == 'mnist' or self.dataset is None:
            # define dataset and dataloader
            mnist = mx.test_utils.get_mnist()

            self.train_iter = mx.io.NDArrayIter(mnist['train_data'],
                                                mnist['train_label'],
                                                self.batch_size, shuffle=True, data_name='data',
                                                label_name='softmax_label')
            self.test_iter = mx.io.NDArrayIter(mnist['test_data'],
                                               mnist['test_label'],
                                               self.batch_size * 2, shuffle=False, data_name='data',
                                               label_name='softmax_label')

        elif self.dataset == 'cifar-10':
            mx.test_utils.get_cifar10()

    def process_log(self):
        # define the paths to the training logs
        logs = [
            (0, self.log_path)
        ]

        # initialize the list of train rank-1 and rank-5 accuracies, along
        # with the training loss
        (trainRank1, trainRank5, trainLoss) = ([], [], [])

        # initialize the list of validation rank-1 and rank-5 accuracies,
        # along with the validation loss
        (valRank1, valRank5, valLoss) = ([], [], [])

        # loop over the training logs
        for (i, (endEpoch, p)) in enumerate(logs):
            # load the contents of the log file, then initialize the batch
            # lists for the training and validation data
            rows = open(p).read().strip()
            (bTrainRank1, bTrainRank5, bTrainLoss) = ([], [], [])
            (bValRank1, bValRank5, bValLoss) = ([], [], [])

            # grab the set of training epochs
            epochs = set(re.findall(r'Epoch\[(\d+)\]', rows))
            epochs = sorted([int(e) for e in epochs])

            # loop over the epochs
            for e in epochs:
                # find all rank-1 accuracies, rank-5 accuracies, and loss
                # values, then take the final entry in the list for each
                s = r'Epoch\[' + str(e) + '\].*accuracy=([0]*\.?[0-9]+)'

                rank1 = re.findall(s, rows)[-2]
                #print(rank1)
                #s = r'Epoch\[' + str(e) + '\].*top_k_accuracy_5=([0]*\.?[0-9]+)'
                #rank5 = re.findall(s, rows)[-2]
                s = r'Epoch\[' + str(e) + '\].*cross-entropy=([0-9]*\.?[0-9]+)'
                loss = re.findall(s, rows)[-2]

                # update the batch training lists
                bTrainRank1.append(float(rank1))
                #bTrainRank5.append(float(rank5))
                bTrainLoss.append(float(loss))

            # extract the validation rank-1 and rank-5 accuracies for each
            # epoch, followed by the loss
            bValRank1 = re.findall(r'Validation-accuracy=(.*)', rows)
            #bValRank5 = re.findall(r'Validation-top_k_accuracy_5=(.*)', rows)
            bValLoss = re.findall(r'Validation-cross-entropy=(.*)', rows)

            # convert the validation rank-1, rank-5, and loss lists to floats
            bValRank1 = [float(x) for x in bValRank1]
            #bValRank5 = [float(x) for x in bValRank5]
            bValLoss = [float(x) for x in bValLoss]
            
            df = pd.DataFrame({'val_acc': bValRank1,
                               'val_loss': bValLoss,
                               'train_acc': bTrainRank1,
                               'train_loss': bTrainLoss})
            
            return df
            
    
    def run_neural(self, update_freq=2, act='relu'):

        # data input and formatting
        data = mx.sym.var('data')  # (bs, 1, 28, 28) - MNIST
        label = mx.sym.var('softmax_label')
        data = mx.sym.Flatten(data)  # (bs, 28*28) - MNIST

        # neural network
        net = mx.sym.FullyConnected(data, num_hidden=10, name='fc1')
        net = mx.sym.Activation(data=net, act_type=act, name='ReLU1')
        net = mx.sym.FullyConnected(net, num_hidden=1000, name='fc2')
        net = mx.sym.Activation(data=net, act_type=act, name='ReLU2')
        net = mx.sym.FullyConnected(net, num_hidden=1000, name='fc3')
        net = mx.sym.Activation(data=net, act_type=act, name='ReLU3')
        net = mx.sym.SoftmaxOutput(net, label=label, name='softmax')
    
        # metrics and eval
        metric_list = [mx.metric.Accuracy(output_names=['softmax_output'], label_names=['softmax_label']),
                       mx.metric.CrossEntropy(output_names=['softmax_output'], label_names=['softmax_label'])]
        eval_metrics = mx.metric.CompositeEvalMetric(metric_list)

        # create and 'compile' network
        if self.is_svrg:
            model = SVRGModule(symbol=net,
                               data_names=['data'],
                               label_names=['softmax_label'],
                               context=self.context,
                               update_freq=update_freq)
        else:
            model = mx.mod.Module(net,
                                  data_names=['data'],
                                  label_names=['softmax_label'],
                                  context=self.context)

        model.bind(data_shapes=self.train_iter.provide_data,
                   label_shapes=self.train_iter.provide_label)
        model.init_params()
        model.init_optimizer(kvstore='local',
                             optimizer=self.optimizer,
                             optimizer_params=self.hyperparams)

        timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
        model.fit(self.train_iter,  # train data
                  eval_data=self.test_iter,  # validation data
                  # optimizer='sgd',  # use SGD to train
                  # optimizer_params = (('learning_rate', lr),),  # use fixed learning rate
                  eval_metric=eval_metrics,  # report accuracy during training
                  # output progress for each 500 data batches
                  batch_end_callback=mx.callback.Speedometer(self.batch_size, 1000),
                  #epoch_end_callback=mx.callback.log_train_metric, #self.log_call,#mx.callback.do_checkpoint('logistic', 10),
                  num_epoch=self.epochs)  # train for at most <epochs> dataset passes
        
        self.df_out = self.process_log()
        print(timestamp)
        self.df_out['timestamp'] = pd.Series(timestamp, index=self.df_out.index)
        self.df_out['optimizer'] = pd.Series(self.optimizer, index=self.df_out.index)
        self.df_out['is_svrg'] = pd.Series(self.is_svrg, index=self.df_out.index)
        self.df_out['batch_size'] = pd.Series(self.batch_size, index=self.df_out.index)
        self.df_out['hyperparams'] = pd.Series(str(self.hyperparams), index=self.df_out.index)
        self.df_out['epoch'] = pd.Series(self.df_out.index + 1, index=self.df_out.index)
        self.df_out['update_freq'] = pd.Series(update_freq, index=self.df_out.index)
        
        #print(self.df_out)
        #val_score = model.score(self.train_iter, 'acc')[0][1]
    
        #train_score = model.score(self.train_iter, 'acc')[0][1]
        #print(val_score, train_score)

        # TODO: Define function returns
        # IDEA: pandas df with all class params, timestamp, and scores. Save this to disk
        # lr_params_svrg.update({lr: [val_score, train_score]})

        #return train_score

In [18]:
nn = NeuralNetwork(dataset='MNIST', 
                   optimizer='sgd', 
                   epochs=10, 
                   context=mx.gpu(), 
                   batch_size=128, 
                   is_svrg=False, 
                   hyperparams={'learning_rate': 0.3})

print(time.time())
nn.run_neural(1, act='relu')
print(time.time())

1544064628.7675602
2018-12-06 00:50:28
1544064637.6647947


In [14]:
# 6s epoch - cpu False
# 100s epoch - cpu True
# 107s epoch - gpu True
# .8s epoch - gpu False

9

In [19]:
nn.df_out

,val_acc,val_loss,train_acc,train_loss,timestamp,optimizer,is_svrg,batch_size,hyperparams,epoch,update_freq
0,0.876172,0.414475,0.491571,1.864575,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},1,1
1,0.897363,0.342930,0.882296,0.387204,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},2,1
2,0.923340,0.252246,0.910065,0.295503,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},3,1
3,0.933496,0.213093,0.927072,0.241143,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},4,1
4,0.939258,0.198226,0.938100,0.205923,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},5,1
5,0.944922,0.179419,0.943863,0.185448,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},6,1
6,0.946777,0.169243,0.948028,0.171785,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},7,1
7,0.947363,0.165610,0.951026,0.161770,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},8,1
8,0.949609,0.161815,0.953258,0.153310,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},9,1
9,0.950781,0.157242,0.955840,0.146524,2018-12-06 00:50:28,sgd,False,128,{'learning_rate': 0.3},10,1


In [ ]:
# MLP EXPERIMENT

run = True

# define hyperparameters
epochs = 25
batch_size = 128

df = pd.DataFrame()

c = 0

if run:
    for _lr in range(1, 10):
        lr = (_lr)/10
        for activation in ['relu', 'tanh']:
            for optimizer in ['sgd', 'adam', 'rmsprop']:
                for is_svrg in [True, False]:
                    
                    #ctx = mx.gpu()
                    print(lr, batch_size, optimizer, is_svrg, c)
                    if is_svrg == False:
                        ctx = mx.gpu()
                        nn = NeuralNetwork(
                            dataset='MNIST', optimizer=optimizer, epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=is_svrg,
                            hyperparams={'learning_rate': lr})
                        nn.run_neural(1, act=activation)
                        c = c + 1
                        df = pd.concat([df, nn.df_out])
                    else:
                        ctx = mx.cpu()
                        for update_freq in [3, 5, 8, 10]:
                            nn = NeuralNetwork(
                                dataset='MNIST', optimizer=optimizer, epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=is_svrg,
                            hyperparams={'learning_rate': lr})
                            nn.run_neural(update_freq, act=activation)
                            print(lr, batch_size, optimizer, is_svrg, update_freq, c)
                            c = c + 1
                                                
                            df = pd.concat([df, nn.df_out])
                
                            df.to_pickle('output_nn.pkl')    
                    #print(lr_svrg_sgd.df_out)

0.1 128 sgd True 0


In [12]:
# LOGIST EXPERIMENT
run = True

# define hyperparameters
#lr = 0.2
#batch_size = 32
epochs = 10
#step = 300

df = pd.DataFrame()

c = 0

if run:
    for _lr in range(1, 10):
        lr = (_lr)/10
        for _batch_size in range(3, 8):
            batch_size = 2**_batch_size
            for optimizer in ['sgd', 'adam', 'rmsprop']:
                for is_svrg in [True, False]:
                    ctx = mx.cpu()
                    #ctx = mx.gpu()
                    print(lr, batch_size, optimizer, is_svrg, c)
                    if is_svrg == False:
                        logreg = LogisticRegression(
                            dataset='MNIST', optimizer=optimizer, epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=is_svrg,
                            hyperparams={'learning_rate': lr})
                        logreg.run_logistic(1)
                        c = c + 1
                        df = pd.concat([df, logreg.df_out])
                    else:
                        for update_freq in [3, 5, 8, 10]:
                            logreg = LogisticRegression(
                                dataset='MNIST', optimizer=optimizer, epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=is_svrg,
                            hyperparams={'learning_rate': lr})
                            logreg.run_logistic(update_freq)
                            print(lr, batch_size, optimizer, is_svrg, update_freq, c)
                            c = c + 1
                            
                    
                            df = pd.concat([df, logreg.df_out])
                
                            df.to_pickle('output.pkl')    
                    #print(lr_svrg_sgd.df_out)

0.1 8 sgd True 0
2018-12-05 08:22:07
0.1 8 sgd True 3 0
2018-12-05 08:24:02
0.1 8 sgd True 5 1
2018-12-05 08:25:55
0.1 8 sgd True 8 2
2018-12-05 08:27:48
0.1 8 sgd True 10 3
0.1 8 sgd False 4
2018-12-05 08:29:41
0.1 8 adam True 5
2018-12-05 08:30:09
0.1 8 adam True 3 5
2018-12-05 08:32:07
0.1 8 adam True 5 6
2018-12-05 08:34:03
0.1 8 adam True 8 7
2018-12-05 08:35:58
0.1 8 adam True 10 8
0.1 8 adam False 9
2018-12-05 08:37:54
0.1 8 rmsprop True 10
2018-12-05 08:38:36
0.1 8 rmsprop True 3 10
2018-12-05 08:40:32
0.1 8 rmsprop True 5 11
2018-12-05 08:42:26
0.1 8 rmsprop True 8 12
2018-12-05 08:44:20
0.1 8 rmsprop True 10 13
0.1 8 rmsprop False 14
2018-12-05 08:46:13
0.1 16 sgd True 15
2018-12-05 08:46:47
0.1 16 sgd True 3 15
2018-12-05 08:47:45
0.1 16 sgd True 5 16
2018-12-05 08:48:42
0.1 16 sgd True 8 17
2018-12-05 08:49:39
0.1 16 sgd True 10 18
0.1 16 sgd False 19
2018-12-05 08:50:36
0.1 16 adam True 20
2018-12-05 08:50:50
0.1 16 adam True 3 20
2018-12-05 08:51:50
0.1 16 adam True 5 21


2018-12-05 10:42:00
0.3 64 rmsprop True 3 175
2018-12-05 10:42:16
0.3 64 rmsprop True 5 176
2018-12-05 10:42:31
0.3 64 rmsprop True 8 177
2018-12-05 10:42:47
0.3 64 rmsprop True 10 178
0.3 64 rmsprop False 179
2018-12-05 10:43:02
0.4 8 sgd True 180
2018-12-05 10:43:09
0.4 8 sgd True 3 180
2018-12-05 10:45:04
0.4 8 sgd True 5 181
2018-12-05 10:46:57
0.4 8 sgd True 8 182
2018-12-05 10:48:50
0.4 8 sgd True 10 183
0.4 8 sgd False 184
2018-12-05 10:50:42
0.4 8 adam True 185
2018-12-05 10:51:11
0.4 8 adam True 3 185
2018-12-05 10:53:09
0.4 8 adam True 5 186
2018-12-05 10:55:05
0.4 8 adam True 8 187
2018-12-05 10:57:02
0.4 8 adam True 10 188
0.4 8 adam False 189
2018-12-05 10:58:57
0.4 8 rmsprop True 190
2018-12-05 10:59:41
0.4 8 rmsprop True 3 190
2018-12-05 11:01:37
0.4 8 rmsprop True 5 191
2018-12-05 11:03:32
0.4 8 rmsprop True 8 192
2018-12-05 11:05:26
0.4 8 rmsprop True 10 193
0.4 8 rmsprop False 194
2018-12-05 11:07:19
0.4 16 sgd True 195
2018-12-05 11:07:49
0.4 16 sgd True 3 195
2018-1

2018-12-05 13:01:01
0.6 64 sgd True 8 347
2018-12-05 13:01:16
0.6 64 sgd True 10 348
0.6 64 sgd False 349
2018-12-05 13:01:31
0.6 64 adam True 350
2018-12-05 13:01:36
0.6 64 adam True 3 350
2018-12-05 13:01:52
0.6 64 adam True 5 351
2018-12-05 13:02:08
0.6 64 adam True 8 352
2018-12-05 13:02:24
0.6 64 adam True 10 353
0.6 64 adam False 354
2018-12-05 13:02:39
0.6 64 rmsprop True 355
2018-12-05 13:02:46
0.6 64 rmsprop True 3 355
2018-12-05 13:03:02
0.6 64 rmsprop True 5 356
2018-12-05 13:03:17
0.6 64 rmsprop True 8 357
2018-12-05 13:03:33
0.6 64 rmsprop True 10 358
0.6 64 rmsprop False 359
2018-12-05 13:03:48
0.7 8 sgd True 360
2018-12-05 13:03:54
0.7 8 sgd True 3 360
2018-12-05 13:05:50
0.7 8 sgd True 5 361
2018-12-05 13:07:43
0.7 8 sgd True 8 362
2018-12-05 13:09:37
0.7 8 sgd True 10 363
0.7 8 sgd False 364
2018-12-05 13:11:29
0.7 8 adam True 365
2018-12-05 13:11:58
0.7 8 adam True 3 365
2018-12-05 13:13:56
0.7 8 adam True 5 366
2018-12-05 13:15:54
0.7 8 adam True 8 367
2018-12-05 13:

2018-12-05 15:18:54
0.9 32 rmsprop True 520
2018-12-05 15:19:05
0.9 32 rmsprop True 3 520
2018-12-05 15:19:35
0.9 32 rmsprop True 5 521
2018-12-05 15:20:04
0.9 32 rmsprop True 8 522
2018-12-05 15:20:33
0.9 32 rmsprop True 10 523
0.9 32 rmsprop False 524
2018-12-05 15:21:03
0.9 64 sgd True 525
2018-12-05 15:21:12
0.9 64 sgd True 3 525
2018-12-05 15:21:28
0.9 64 sgd True 5 526
2018-12-05 15:21:43
0.9 64 sgd True 8 527
2018-12-05 15:21:58
0.9 64 sgd True 10 528
0.9 64 sgd False 529
2018-12-05 15:22:13
0.9 64 adam True 530
2018-12-05 15:22:18
0.9 64 adam True 3 530
2018-12-05 15:22:34
0.9 64 adam True 5 531
2018-12-05 15:22:50
0.9 64 adam True 8 532
2018-12-05 15:23:05
0.9 64 adam True 10 533
0.9 64 adam False 534
2018-12-05 15:23:21
0.9 64 rmsprop True 535
2018-12-05 15:23:27
0.9 64 rmsprop True 3 535
2018-12-05 15:23:43
0.9 64 rmsprop True 5 536
2018-12-05 15:23:58
0.9 64 rmsprop True 8 537
2018-12-05 15:24:13
0.9 64 rmsprop True 10 538
0.9 64 rmsprop False 539
2018-12-05 15:24:28


In [ ]:
if __name__ == '__main__':
    try:
        from svrg_optimization.svrg_module import SVRGModule
    except ModuleNotFoundError:
        download_svrg_module()
        from svrg_optimization.svrg_module import SVRGModule

    run = True

    # define hyperparameters
    lr = 0.2
    batch_size = 32
    epochs = 2
    step = 300

    if run:
        ctx = mx.cpu()
        lr_svrg_sgd = LogisticRegression(
            dataset='MNIST', optimizer='sgd', epochs=epochs, context=ctx, batch_size=batch_size, is_svrg=True,
            hyperparams={'learning_rate': 0.2})
        lr_svrg_sgd.run_logistic(2)